### Preparação

In [ ]:
!pip install -q yfinance

     |████████████████████████████████| 6.3MB 8.8MB/s 


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
def fix_col_names(df):
  return ['IBOV' if col =='^BVSP' else col.rstrip('.SA') for col in df.columns]

In [ ]:
tickers = ['BBDC4.SA','BBAS3.SA','SANB4.SA','ABCB4.SA','ITSA4.SA','BBSE3.SA','TRPL4.SA','EGIE3.SA','TAEE3.SA','IRBR3.SA','JHSF3.SA','FLRY3.SA','LEVE3.SA','SAPR4.SA','AESB3.SA']
numero_ativos = len(tickers)

prices = yf.download(tickers, period='1y')['Adj Close']
prices.dropna(inplace=True)
prices.columns = fix_col_names(prices)

[*********************100%***********************]  15 of 15 completed


In [ ]:
tickers

In [ ]:
retorno = np.log(prices / prices.shift(1)).dropna()

In [ ]:
retorno.head()

,ABCB4,AESB3,BBAS3,BBDC4,BBSE3,EGIE3,FLRY3,IRBR3,ITSA4,JHSF3,LEVE3,SANB4,SAPR4,TAEE3,TRPL4
Date,,,,,,,,,,,,,,,
2021-03-30,0.016990,0.044503,0.030406,0.016584,0.029071,0.009186,0.026510,0.013072,0.014430,0.059138,0.042711,0.009028,0.015855,0.016273,0.001141
2021-03-31,-0.023178,-0.016529,-0.008828,-0.021800,-0.021620,0.006476,-0.000391,-0.004882,-0.014430,-0.025533,0.005213,-0.015253,-0.006764,-0.008491,-0.006847
2021-04-01,-0.016728,-0.023257,-0.016557,-0.037295,-0.013702,-0.008402,0.003509,-0.013136,-0.021548,0.011901,-0.010015,-0.025786,0.004515,-0.025914,0.000000
2021-04-05,0.023911,-0.014815,-0.001336,0.010040,0.006251,0.010552,0.008527,0.003300,0.012789,0.025411,0.031448,-0.020917,-0.015891,0.017351,0.005904
2021-04-06,-0.000695,-0.013221,-0.012109,-0.015889,-0.000416,-0.001193,0.038241,0.001646,-0.009823,0.018965,0.013061,0.002513,-0.002291,0.008564,-0.003145


## 1. Simulação de Monte Carlo para pesos

In [ ]:
# Cria uma matriz de números aleatórios
numero_carteiras = 100000
np.random.seed(101) # Essa linha garante que sejam gerados os mesmo números. Remover para que sejam gerados números aleatórios
matriz_aleatoria = np.random.rand(numero_carteiras,numero_ativos)
matriz_aleatoria

array([[0.51639863, 0.57066759, 0.02847423, ..., 0.1818924 , 0.78560176,
        0.96548322],
       [0.23235366, 0.08356143, 0.60354842, ..., 0.57878954, 0.73481906,
        0.54196177],
       [0.91315356, 0.80792015, 0.40299783, ..., 0.7015073 , 0.89047987,
        0.1595603 ],
       ...,
       [0.18060186, 0.17492533, 0.60011791, ..., 0.68886193, 0.16845854,
        0.97295756],
       [0.70253754, 0.79824655, 0.93006823, ..., 0.4041766 , 0.97368789,
        0.40276172],
       [0.69438184, 0.72268859, 0.60838055, ..., 0.22428272, 0.55479503,
        0.39207524]])

In [ ]:
matriz_aleatoria.sum(axis=1, keepdims=True)

array([[7.75763499],
       [6.87174377],
       [8.88535396],
       ...,
       [6.62243474],
       [8.14648485],
       [8.25231244]])

In [ ]:
# Cria a matriz de pesos através da normalização da matriz aleatória
pesos =  matriz_aleatoria / matriz_aleatoria.sum(axis=1, keepdims=True)
pesos

array([[0.0665665 , 0.07356206, 0.00367048, ..., 0.02344689, 0.1012682 ,
        0.12445587],
       [0.03381291, 0.01216015, 0.08783046, ..., 0.08422746, 0.10693342,
        0.07886816],
       [0.10277064, 0.09092718, 0.04535529, ..., 0.07895097, 0.10021884,
        0.01795768],
       ...,
       [0.02727122, 0.02641405, 0.09061892, ..., 0.10401944, 0.02543755,
        0.14691841],
       [0.08623812, 0.09798662, 0.11416804, ..., 0.04961362, 0.11952246,
        0.04943994],
       [0.08414391, 0.08757407, 0.07372243, ..., 0.02717817, 0.06722904,
        0.04751095]])

In [ ]:
retorno.dot(pesos.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,99960,99961,99962,99963,99964,99965,99966,99967,99968,99969,99970,99971,99972,99973,99974,99975,99976,99977,99978,99979,99980,99981,99982,99983,99984,99985,99986,99987,99988,99989,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-30,0.019406,0.021649,0.025602,0.025529,0.025628,0.023847,0.023083,0.023808,0.026556,0.019796,0.023826,0.019835,0.024364,0.025494,0.022091,0.020136,0.022799,0.024420,0.024607,0.024660,0.021432,0.024247,0.022812,0.026712,0.022830,0.020399,0.023889,0.024986,0.017251,0.021882,0.026753,0.022103,0.020070,0.025708,0.022290,0.021394,0.027718,0.022615,0.025942,0.022863,...,0.026431,0.021776,0.024871,0.026766,0.025466,0.022364,0.028571,0.025571,0.026226,0.019947,0.022309,0.019587,0.026105,0.022052,0.028311,0.026472,0.021020,0.021304,0.027909,0.020139,0.020217,0.024774,0.023902,0.022676,0.025237,0.027475,0.025283,0.026696,0.024570,0.024375,0.023768,0.022827,0.023916,0.025533,0.022627,0.025414,0.021150,0.018347,0.024307,0.024120
2021-03-31,-0.009223,-0.007763,-0.012877,-0.012803,-0.010452,-0.010720,-0.008488,-0.009615,-0.011521,-0.012206,-0.010710,-0.009404,-0.012740,-0.011462,-0.011662,-0.010597,-0.012784,-0.009045,-0.009777,-0.011348,-0.009867,-0.012303,-0.008252,-0.010806,-0.010155,-0.011203,-0.008767,-0.009046,-0.010043,-0.011791,-0.010344,-0.009455,-0.009002,-0.010127,-0.010426,-0.009476,-0.012007,-0.009907,-0.013451,-0.009604,...,-0.012761,-0.008568,-0.010219,-0.011129,-0.010324,-0.011215,-0.011518,-0.012194,-0.011557,-0.010539,-0.007036,-0.010396,-0.012041,-0.012148,-0.009591,-0.006990,-0.009608,-0.010829,-0.011684,-0.012336,-0.010100,-0.009120,-0.010384,-0.010137,-0.011060,-0.012937,-0.014536,-0.012031,-0.010198,-0.014558,-0.011093,-0.012788,-0.012710,-0.011434,-0.010673,-0.011246,-0.007987,-0.008255,-0.010151,-0.010081
2021-04-01,-0.013315,-0.013378,-0.011482,-0.013699,-0.012259,-0.010472,-0.010697,-0.010508,-0.015104,-0.014205,-0.011445,-0.012733,-0.011272,-0.009127,-0.010559,-0.014630,-0.015085,-0.012925,-0.015136,-0.010128,-0.012294,-0.013928,-0.012574,-0.014921,-0.008542,-0.012600,-0.011678,-0.010436,-0.014697,-0.010727,-0.010418,-0.012214,-0.014434,-0.010514,-0.012973,-0.011748,-0.007976,-0.010685,-0.014700,-0.010924,...,-0.011628,-0.010921,-0.010657,-0.010664,-0.012643,-0.011363,-0.009623,-0.013770,-0.012130,-0.012971,-0.011647,-0.013821,-0.011224,-0.017535,-0.010018,-0.009821,-0.012563,-0.015753,-0.008437,-0.014730,-0.012865,-0.011674,-0.013629,-0.012999,-0.010963,-0.014396,-0.013521,-0.011489,-0.013134,-0.016822,-0.011087,-0.013892,-0.014203,-0.011620,-0.012282,-0.010093,-0.011151,-0.008868,-0.015092,-0.013710
2021-04-05,0.008351,0.008442,0.007775,0.005407,0.005266,0.006994,0.007196,0.009841,0.006971,0.005447,0.004941,0.003782,0.010076,0.009209,0.004921,0.009643,0.005108,0.005928,0.004819,0.006346,0.006186,0.005785,0.006861,0.009898,0.007393,0.005463,0.007672,0.006497,0.008348,0.006771,0.009396,0.009993,0.010904,0.008604,0.007215,0.010939,0.010510,0.010700,0.007492,0.009751,...,0.007636,0.003798,0.004840,0.007705,0.010280,0.004427,0.010420,0.009448,0.008535,0.008559,0.008451,0.009820,0.001876,0.003383,0.006760,0.008495,0.003894,0.006848,0.010154,0.000455,0.008032,0.008301,0.009781,0.006516,0.008025,0.009789,0.006437,0.007604,0.008817,0.003300,0.009911,0.006250,0.007823,0.008779,0.010379,0.007126,0.010234,0.005145,0.006500,0.007113
2021-04-06,0.001143,0.002698,0.003491,0.001480,0.000563,0.001765,0.000891,0.005183,0.000911,-0.001136,0.004131,0.002230,0.002659,0.003092,0.001123,0.001984,-0.000681,0.002522,-0.002958,0.002911,0.003833,-0.000276,0.002675,0.000952,0.002868,-0.002460,0.002401,0.003955,-0.000837,0.001954,0.003836,-0.000926,-0.001156,0.001158,0.002141,0.001906,0.007923,0.006549,0.000744,0.005853,...,0.003923,0.004635,0.001850,0.003261,0.001557,0.003641,0.001833,0.001805,0.00

In [ ]:
# Calcula o retorno diário da cada carteira
carteiras = retorno.dot(pesos.T)


In [ ]:
carteiras.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,99960,99961,99962,99963,99964,99965,99966,99967,99968,99969,99970,99971,99972,99973,99974,99975,99976,99977,99978,99979,99980,99981,99982,99983,99984,99985,99986,99987,99988,99989,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-05-13,0.017044,0.017590,0.019246,0.020153,0.017753,0.018486,0.016661,0.018010,0.018395,0.019327,0.018709,0.018608,0.018640,0.020208,0.020354,0.018973,0.021538,0.018717,0.019454,0.017851,0.019928,0.018608,0.014412,0.016402,0.019161,0.019290,0.016902,0.018104,0.017123,0.019152,0.018168,0.020022,0.016366,0.015514,0.018606,0.018902,0.019356,0.020517,0.021553,0.019400,...,0.016494,0.019810,0.016722,0.016947,0.015926,0.018641,0.019155,0.018446,0.017697,0.021222,0.020113,0.018844,0.019903,0.017040,0.015648,0.019041,0.021441,0.019768,0.018673,0.021758,0.018865,0.017161,0.019655,0.017673,0.016530,0.017711,0.020969,0.017420,0.019043,0.019779,0.020557,0.018965,0.018322,0.018670,0.019541,0.018328,0.017687,0.018078,0.018255,0.019555
2021-05-14,0.019049,0.034626,0.010221,0.013484,0.024856,0.009898,0.021226,0.021740,0.017246,0.014827,0.014246,0.019514,0.017768,0.016037,0.011761,0.029500,0.008243,0.017594,0.014822,0.012880,0.021477,0.016870,0.025320,0.026618,0.018047,0.007314,0.014978,0.022621,0.018987,0.011609,0.019501,0.020268,0.026866,0.020195,0.026561,0.016431,0.019640,0.028411,0.010427,0.014683,...,0.011742,0.016301,0.019076,0.022749,0.026155,0.017213,0.027010,0.018776,0.020689,0.012940,0.017796,0.023572,0.011312,0.017163,0.025279,0.021614,0.010698,0.021393,0.017925,0.007144,0.019411,0.027755,0.023106,0.026062,0.019998,0.021394,0.014349,0.020063,0.021989,0.018169,0.019105,0.011975,0.028969,0.019246,0.016652,0.016560,0.023840,0.012219,0.031369,0.020124
2021-05-17,0.012854,0.023249,0.010208,0.011915,0.020641,0.013176,0.018768,0.018065,0.015241,0.011214,0.014814,0.010256,0.018708,0.019595,0.010786,0.020477,0.008126,0.016567,0.015638,0.013722,0.014603,0.015681,0.017562,0.021713,0.014828,0.008556,0.013650,0.014799,0.010465,0.012641,0.020210,0.016729,0.019722,0.019248,0.016866,0.015562,0.018079,0.019735,0.013736,0.012268,...,0.012599,0.013385,0.017700,0.018522,0.020201,0.013233,0.026119,0.017832,0.020628,0.008368,0.011278,0.018464,0.014518,0.012838,0.021580,0.023126,0.006840,0.017195,0.020416,0.007031,0.014214,0.022148,0.015258,0.017872,0.015146,0.018383,0.016013,0.018793,0.018000,0.013386,0.014459,0.009978,0.020475,0.018309,0.017212,0.015014,0.016958,0.007658,0.021045,0.018030
2021-05-18,0.006816,0.008974,0.006603,0.006985,0.009754,0.007070,0.007323,0.006015,0.006336,0.008010,0.006689,0.005609,0.010608,0.009772,0.009820,0.011409,0.006928,0.007344,0.009096,0.003139,0.008569,0.005830,0.006396,0.011383,0.007136,0.006372,0.004330,0.004952,0.006209,0.009127,0.010207,0.008639,0.008780,0.006600,0.010058,0.009925,0.005303,0.008638,0.008220,0.004593,...,0.005424,0.003792,0.006499,0.006509,0.006934,0.005232,0.012337,0.010203,0.003917,0.006953,0.003688,0.010631,0.006029,0.007136,0.006440,0.010101,0.004832,0.012258,0.007535,0.009356,0.008419,0.011106,0.005791,0.009595,0.005126,0.009371,0.009277,0.006461,0.010084,0.007803,0.004975,0.009157,0.010892,0.006386,0.010039,0.009149,0.007350,0.003655,0.011013,0.010277
2021-05-19,0.007224,0.008373,0.005774,0.007483,0.007700,0.007416,0.007942,0.007549,0.005875,0.007102,0.008168,0.008040,0.006287,0.007184,0.007152,0.007588,0.005591,0.007779,0.006215,0.007173,0.007662,0.006636,0.007514,0.005600,0.007964,0.006634,0.007404,0.007513,0.006261,0.006849,0.006283,0.006598,0.007207,0.007151,0.007389,0.006713,0.007394,0.008511,0.006459,0.007370,...,0.006605,0.008731,0.007984,0.006893,0.007125,0.008536,0.006529,0.005958,0.007371,0.006623,0.007496,0.006991,0.007743,0.006487,0.007732,0.007889,0.007302,0.006932,0.007177,0.006813,0.007470,0.007836,0.007534,0.007190,0.007178,0.005512,0.006942,0

In [ ]:
# Calcula o retorno e volatilidade anualizados das carteiras
carteiras_aleatorias = pd.DataFrame()
carteiras_aleatorias['retorno'] = carteiras.mean() * 252
carteiras_aleatorias['volatilidade'] = carteiras.std() * np.sqrt(252)

In [ ]:
carteiras_aleatorias

,retorno,volatilidade
0,0.372475,0.154433
1,0.645226,0.183724
2,0.333669,0.154282
3,0.266823,0.179476
4,0.464183,0.178555
...,...,...
99995,0.376057,0.163254
99996,0.537968,0.157775
99997,0.208191,0.145675
99998,0.571027,0.182495


In [ ]:
# Calcula o retorno anualizado dos ativos
ativos = pd.DataFrame()
ativos['retorno'] = retorno.mean() * 252
ativos['volatilidade'] = retorno.std() * np.sqrt(252)

In [ ]:
ativos

,retorno,volatilidade
ABCB4,1.143453,0.234468
AESB3,-0.981063,0.429613
BBAS3,0.573333,0.195567
BBDC4,0.292802,0.292824
BBSE3,-0.194033,0.218808
EGIE3,0.007913,0.176570
FLRY3,0.504443,0.224701
IRBR3,0.023645,0.280993
ITSA4,0.257028,0.231108
JHSF3,1.331490,0.300125


In [ ]:
# Gráfico
fig = go.Figure()

fig.add_scatter(y=carteiras_aleatorias['retorno'], 
                x=carteiras_aleatorias['volatilidade'], 
                mode='markers')

fig.add_scatter(x=ativos['volatilidade'], 
                y=ativos['retorno'], 
                marker=dict(color='red', size=7),
                text=ativos.index, 
                mode='markers+text')

fig.layout.xaxis.title = 'Volatilidade (desvio-padrão anualizado)'
fig.layout.yaxis.title = 'Retorno médio anualizado'
fig.layout.title = 'Risco-Retorno'

fig.update_traces(textposition='top center')
fig.update_layout(autosize=False, showlegend=False)


fig.show()

Output hidden; open in https://colab.research.google.com to view.

## 2. Sharpe-Ratio (Índice de Sharpe)

$$ SR = \frac{R_p - R_f}{\sigma_p}$$

sendo:

$SR$: Sharpe Ratio 

$R_p$: Retorno do portfolio

$R_f$: Retorno do ativo livre de risco

$\sigma_p$: Desvio-padrão do portfolio

In [ ]:
Rf = 0.02
sigmaP = 0

In [ ]:
# Calcula o Índice Sharpe das carteiras
carteiras_aleatorias['sharpe'] = (carteiras_aleatorias['retorno'] - Rf) / carteiras_aleatorias['volatilidade']

In [ ]:
carteiras_aleatorias

,retorno,volatilidade,sharpe
0,0.372475,0.154433,2.282383
1,0.645226,0.183724,3.403074
2,0.333669,0.154282,2.033086
3,0.266823,0.179476,1.375244
4,0.464183,0.178555,2.487650
...,...,...,...
99995,0.376057,0.163254,2.180998
99996,0.537968,0.157775,3.282949
99997,0.208191,0.145675,1.291851
99998,0.571027,0.182495,3.019409


In [ ]:
# Encontra a carteira com maior Índice Sharpe
id_max_sr = carteiras_aleatorias['sharpe'].idxmax()
max_sr = carteiras_aleatorias.iloc[id_max_sr]

In [ ]:
max_sr

retorno         0.933612
volatilidade    0.181452
sharpe          5.034994
Name: 94079, dtype: float64

In [ ]:
# Calcula o Índice Sharpe dos ativos
ativos['sharpe'] = (ativos['retorno'] - Rf) / ativos['volatilidade']

In [ ]:
ativos

In [ ]:
# Gráfico
fig = go.Figure()

fig.add_scatter(y=carteiras_aleatorias['retorno'], 
                x=carteiras_aleatorias['volatilidade'], 
                mode='markers',
                showlegend=False,
                marker=dict(color=carteiras_aleatorias['sharpe'], 
                            size=7, 
                            colorscale="Inferno", 
                            colorbar=dict(title="Sharpe")))

fig.add_scatter(x=[max_sr['volatilidade']], 
                y=[max_sr['retorno']], 
                marker=dict(color='red', size=9, symbol='star'),
                text='Máx SR',
                showlegend=False, 
                mode='markers+text')
'''
fig.add_scatter(x=[sigmaP], 
                y=[Rf], 
                marker=dict(color='red', size=9),
                text='RF',
                showlegend=False, 
                mode='markers+text')

fig.add_scatter(x=[sigmaP, max_sr['volatilidade']], 
                y=[Rf, max_sr['retorno']], 
                marker=dict(color='red', size=9),
                text='RF',
                showlegend=False, 
                mode='lines')'''

fig.add_scatter(x=ativos['volatilidade'], 
                y=ativos['retorno'], 
                showlegend=False,
                marker=dict(color='blue', size=7),
                text=ativos.index, 
                mode='markers+text')

fig.layout.xaxis.title = 'Volatilidade'
fig.layout.yaxis.title = 'Retorno Esperado'
fig.layout.title = 'Risco-Retorno'

fig.update_traces(textfont_size=12, 
                  textposition='middle left', 
                  textfont_color='black',
                  hovertemplate='<b>Retorno: </b> %{y:.1%}'+
                                '<br><b>Volatilidade:</b> %{x:.1%}'+
                                '<br><b>Sharpe:</b> %{marker.color:.2f} ')

fig.update_layout(autosize=False)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

## 3. Fronteira Eficiente

In [81]:
from scipy.optimize import minimize

In [82]:
def calcula_portfolio_vol(pesos, cov):
    return (pesos.T @ cov @ pesos)**0.5

In [83]:
def calcula_portfolio_ret(pesos, retornos):
    return pesos.T @ retornos

In [84]:
# Função que calcula a vol mínima dada um nível de retorno, o retorno esperado e a matriz de covariância
def vol_minima(nivel_de_retorno, re, cov):  
    aposta_inicial = np.repeat(1/numero_ativos, numero_ativos) # Aposta Inicial = Pesos Iguais
    limites = ((0.0, 1.0),) * numero_ativos 
    
    soma_de_pesos_1 = {'type': 'eq',
                        'fun': lambda pesos: np.sum(pesos) - 1
    }
    retorno_alvo = {'type': 'eq',
                    'args': (re,),
                     'fun': lambda pesos, re: nivel_de_retorno - calcula_portfolio_ret(pesos,re)
    }
    pesos = minimize(calcula_portfolio_vol, aposta_inicial,
                     args=(cov,), 
                     method='SLSQP',
                     options={'disp': False},
                     constraints=(soma_de_pesos_1, retorno_alvo),
                     bounds=limites)
    return pesos['x']

In [85]:
def fronteira_eficiente(n_pontos, re, cov):    
    faixa_de_retorno = np.linspace(re.min(), re.max(), n_pontos)
    pesos_otimos = [vol_minima(nivel_de_retorno, re, cov) for nivel_de_retorno in faixa_de_retorno]
    
    #pesos = calcula_pesos_otimos(n_pontos, re, cov)
    rets = [calcula_portfolio_ret(pesos, re) for pesos in pesos_otimos]
    vols = [calcula_portfolio_vol(pesos, cov) for pesos in pesos_otimos]
    fe = pd.DataFrame({
        "Retornos": rets, 
        "Volatilidade": vols
    })
    return fe

In [86]:
fe = fronteira_eficiente(100, retorno.mean()*252, retorno.cov()*252)

In [87]:
# Gráfico
fig = px.line(fe, x='Volatilidade', y='Retornos')

fig.add_scatter(y=carteiras_aleatorias['retorno'], 
                x=carteiras_aleatorias['volatilidade'], 
                mode='markers',
                showlegend=False,
                marker=dict(color=carteiras_aleatorias['sharpe'], 
                            size=3, 
                            colorscale="inferno_r", 
                            colorbar=dict(title="Índice Sharpe")))

fig.add_scatter(x=[max_sr['volatilidade']], 
                y=[max_sr['retorno']], 
                marker=dict(color='red', size=9, symbol='star'),
                text='Máx SR',
                showlegend=False, 
                mode='markers+text')

fig.add_scatter(x=ativos['volatilidade'], 
                y=ativos['retorno'], 
                showlegend=False,
                marker=dict(color='red', size=7),
                text=ativos.index, 
                mode='markers+text')

fig.layout.xaxis.title = 'Volatilidade'
fig.layout.yaxis.title = 'Retorno Esperado'
fig.layout.title = 'Risco-Retorno'

fig.update_traces(line=dict(color="darkblue", width=4),
                  textfont_size=12, 
                  textposition='top center', 
                  textfont_color='black',
                  hovertemplate='<b>Retorno: </b> %{y:.1%}'+
                                '<br><b>Volatilidade:</b> %{x:.1%}'+
                                '<br><b>Sharpe:</b> %{marker.color:.2f} ' )

fig.update_layout(autosize=False)

fig.show()

Output hidden; open in https://colab.research.google.com to view.